In [ ]:
# default_exp models.esrnn.esrnn_model

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#export
from typing import Union, List

import numpy as np
import torch as t
import torch.nn as nn
import pytorch_lightning as pl

from nixtla.models.esrnn.esrnn_model import _ES, _ESI, _MedianResidual, _ESM, _RNN, _ESRNN
from nixtla.losses.pytorch import (
    MAPELoss, MASELoss, SMAPELoss, 
    MSELoss, MAELoss, SmylLoss, 
    PinballLoss, MQLoss, wMQLoss 
)

In [ ]:
#export
class ESRNN(pl.LightningModule):
    def __init__(self, n_series: int,
                  n_t: int, n_s: int,
                  input_size: int, output_size: int,
                  es_component: str = 'multiplicative', 
                  cell_type: str = 'LSTM', state_hsize: int = 50, 
                  dilations: List[List[int]] = [[1, 2], [4, 8]], 
                  add_nl_layer: bool = False, seasonality: List[int] = [],
                  learning_rate: float = 1e-3, lr_scheduler_step_size: int = 9,
                  lr_decay: float = 0.9, per_series_lr_multip: float = 1.,
                  gradient_eps: float = 1e-8, 
                  gradient_clipping_threshold: float = 20.,
                  rnn_weight_decay: float = 0., noise_std: float = 1e-3,
                  level_variability_penalty: float = 20.,
                  testing_percentile: Union[int, List] = 50, 
                  training_percentile: Union[int, List] = 50,
                  loss: str = 'SMYL', val_loss: str = 'MAE'):
        super(ESRNN, self).__init__()

        #------------------------ Model Attributes ------------------------#
        # Architecture parameters
        self.n_series = n_series
        self.n_t = n_t
        self.n_s = n_s 
        self.input_size = input_size
        self.output_size = output_size
        self.es_component = es_component
        self.cell_type = cell_type
        self.state_hsize = state_hsize
        self.dilations = dilations
        self.add_nl_layer = add_nl_layer
        self.seasonality = seasonality

        # Regularization and optimization parameters
        self.learning_rate = learning_rate
        self.lr_scheduler_step_size = lr_scheduler_step_size
        self.lr_decay = lr_decay
        self.per_series_lr_multip = per_series_lr_multip
        self.gradient_eps = gradient_eps
        self.gradient_clipping_threshold = gradient_clipping_threshold
        self.rnn_weight_decay = rnn_weight_decay
        self.noise_std = noise_std
        self.level_variability_penalty = level_variability_penalty
        self.testing_percentile = testing_percentile
        self.training_percentile = training_percentile
        self.loss = loss
        self.val_loss = val_loss
        self.loss_fn = self.__loss_fn(loss)
        self.val_loss_fn = self.__loss_fn(val_loss)

        # MQESRNN
        self.mq = isinstance(self.training_percentile, list)
        self.output_size_m = len(self.training_percentile) if self.mq else 1

        #Defining model
        self.esrnn = _ESRNN(n_series=self.n_series, 
                            input_size=self.input_size,
                            output_size=self.output_size, 
                            output_size_m=self.output_size_m,
                            n_t=self.n_t, n_s=self.n_s,
                            es_component=self.es_component, 
                            seasonality=self.seasonality,
                            noise_std=self.noise_std, 
                            cell_type=self.cell_type,
                            dilations=self.dilations, 
                            state_hsize=self.state_hsize,
                            add_nl_layer=self.add_nl_layer, 
                            device=self.device).to(self.device)
            
    def forward(self):
        pass
    
    def __loss_fn(self, loss_name: str):
        #TODO: replace with kwargs
        def loss(forecast, target, mask=None, x=None, levels=None):
            if loss_name == 'SMYL':
                return SmylLoss(y=target, y_hat=forecast, levels=levels, mask=mask,
                                tau=(self.training_percentile / 100),
                                level_variability_penalty=self.level_variability_penalty)
            elif loss_name == 'MQ':
                quantiles = [tau / 100 for tau in self.training_percentile]
                quantiles = self.to_tensor(np.array(quantiles))
                return MQLoss(y=target, y_hat=forecast, quantiles=quantiles, mask=mask)
            elif loss_name == 'wMQ':
                quantiles = [tau / 100 for tau in self.training_percentile]
                quantiles = self.to_tensor(np.array(quantiles))
                return wMQLoss(y=target, y_hat=forecast, quantiles=quantiles, mask=mask)
            elif loss_name == 'MAPE':
                return MAPELoss(y=target, y_hat=forecast, mask=mask)
            elif loss_name == 'MASE':
                return MASELoss(y=target, y_hat=forecast, y_insample=x, seasonality=loss_hypar, mask=mask)
            elif loss_name == 'SMAPE':
                return SMAPELoss(y=target, y_hat=forecast, mask=mask)
            elif loss_name == 'MSE':
                return MSELoss(y=target, y_hat=forecast, mask=mask)
            elif loss_name == 'MAE':
                return MAELoss(y=target, y_hat=forecast, mask=mask)
            elif loss_name == 'PINBALL':
                return PinballLoss(y=target, y_hat=forecast, mask=mask, 
                                   tau=(self.training_percentile/100))
            else:
                raise Exception(f'Unknown loss function: {loss_name}')
        return loss

    def training_step(self, batch, batch_idx):
        #Parsing batch
        S = t.tensor(batch['S'])
        Y = t.tensor(batch['Y'])
        X = t.tensor(batch['X'])
        idxs = t.tensor(batch['idxs'], dtype=t.long)
        step_size = 24 # to be defined
        outsample_y, forecast, levels = self.esrnn(S=S, Y=Y, X=X, idxs=idxs,
                                                   step_size=step_size)
        
        outsample_mask = t.ones_like(outsample_y)
        loss = self.loss_fn(x=Y,
                            forecast=forecast,
                            target=outsample_y,
                            mask=outsample_mask, 
                            levels=levels) 

        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, idx):
        S = t.tensor(batch['S'])
        Y = t.tensor(batch['Y'])
        X = t.tensor(batch['X'])
        idxs = t.tensor(batch['idxs'], dtype=t.long)
        step_size = 24
        
        target, forecast = self.esrnn.predict(S=S, Y=Y, X=X, idxs=idxs,
                                              step_size=step_size)
        outsample_mask = t.ones_like(target)
        loss = self.val_loss_fn(forecast=forecast,
                                target=target,
                                mask=outsample_mask)
        self.log('val_loss', loss)
        
        return loss
        
    
    def configure_optimizers(self):
        optimizer = t.optim.Adam(self.esrnn.parameters(), lr=1e-3)
        return optimizer

## Pruebas

In [ ]:
from nixtla.data.datasets.epf import EPF, EPFInfo
from nixtla.data.tsdataset import TimeSeriesDataset
from nixtla.data.tsloader import TimeSeriesLoader

Y_df, X_df, _ = EPF.load(directory='./data', group=EPFInfo.groups[0])

X_df = X_df[['unique_id', 'ds', 'week_day']]

# Leveling Y_df (multiplicative model)
Y_min = Y_df.y.min()
Y_df.y = Y_df.y - Y_min + 20

train_ts_dataset = TimeSeriesDataset(Y_df=Y_df, S_df=None, X_df=X_df,
                                     ds_in_test=728*24, verbose=False)

outsample_ts_dataset = TimeSeriesDataset(Y_df=Y_df, S_df=None, X_df=X_df,
                                         ds_in_test=728*24, is_test=True, verbose=False)

train_ts_loader = TimeSeriesLoader(ts_dataset=train_ts_dataset,
                                   model='esrnn',
                                   window_sampling_limit= 500_000, # To limit backprop time
                                   input_size=7*24,
                                   output_size=24,
                                   idx_to_sample_freq=24,
                                   len_sample_chunks=3*7*24,
                                   complete_inputs=True,
                                   batch_size=32,
                                   shuffle=False)

val_ts_loader = TimeSeriesLoader(ts_dataset=outsample_ts_dataset,
                                 model='esrnn',
                                 window_sampling_limit= 500_000, 
                                 input_size=7*24,
                                 output_size=24,
                                 idx_to_sample_freq=24,
                                 len_sample_chunks=4*7*24,
                                 complete_inputs=False,
                                 batch_size=1,
                                 shuffle=False)

In [ ]:
esrnn = ESRNN(n_series=1, n_t=1, n_s=0,
              input_size=5*24,
              output_size=24,
              seasonality=[24],
              noise_std=0.01,
              es_component='multiplicative',
              cell_type='LSTM',
              state_hsize=50,
              dilations=[[1, 2], [7, 14]],
              add_nl_layer=False)

In [ ]:
trainer = pl.Trainer(max_epochs=5)
trainer.fit(esrnn, train_ts_loader, val_ts_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name  | Type   | Params
---------------------------------
0 | esrnn | _ESRNN | 125 K 
---------------------------------
125 K     Trainable params
0         Non-trainable params
125 K     Total params
0.503     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/federicogarza2/anaconda3/envs/nixtla/lib/python3.7/site-packages/ipykernel_launcher.py:128: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/federicogarza2/anaconda3/envs/nixtla/lib/python3.7/site-packages/ipykernel_launcher.py:129: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Training: 0it [00:00, ?it/s]

/home/federicogarza2/anaconda3/envs/nixtla/lib/python3.7/site-packages/ipykernel_launcher.py:109: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/federicogarza2/anaconda3/envs/nixtla/lib/python3.7/site-packages/ipykernel_launcher.py:110: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [ ]:
trainer.logged_metrics

{'val_loss': tensor(3.5918), 'epoch': tensor(4.), 'train_loss': tensor(0.0392)}